In [1]:
!pip install python-dotenv
!pip install srai[all]
!pip install folium matplotlib mapclassify

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 kB 6.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 543.2/543.2 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.2/118.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [21]:
!pip install pgvector  # Install the pgvector module
!pip install geoalchemy2  # Install the geoalchemy2 module

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.2 MB/s eta 0:00:00


In [35]:
import pandas as pd
from sqlalchemy import create_engine, Table, Column, Integer, Float, MetaData
from sqlalchemy.dialects.postgresql import ARRAY
import geopandas as gpd
from pgvector.sqlalchemy import Vector
from geoalchemy2 import Geometry
from sqlalchemy import text, insert
import numpy as np

In [2]:
from srai.embedders import Hex2VecEmbedder
from srai.joiners import IntersectionJoiner
from srai.loaders import OSMPbfLoader
from srai.loaders.osm_loaders.filters import HEX2VEC_FILTER
from srai.neighbourhoods.h3_neighbourhood import H3Neighbourhood
from srai.regionalizers import H3Regionalizer, geocode_to_region_gdf
from srai.plotting import plot_regions, plot_numeric_data

loader = OSMPbfLoader()
regionalizer = H3Regionalizer(resolution=11)
joiner = IntersectionJoiner()

area = geocode_to_region_gdf("City of London")
features = loader.load(area, HEX2VEC_FILTER)
regions = regionalizer.transform(area)
joint = joiner.transform(regions, features)

/usr/local/lib/python3.10/dist-packages/srai/loaders/osm_loaders/osm_pbf_loader.py:128: FutureWarning: Use `convert_geometry_to_geodataframe` instead. Deprecated since 0.8.1 version.
  features_gdf = pbf_reader.get_features_gdf_from_geometry(
100%|███████████████████████████████████████| 106M/106M [00:00<00:00, 18.6GB/s]
SHA256 hash of downloaded file: 1fb478edf102cd437f2786277c510b3951c0edae85ceef1499c3051ca44b5062
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.


Output()

Finished operation in 0:03:16

In [3]:
embedder = Hex2VecEmbedder()
neighbourhood = H3Neighbourhood(regions_gdf=regions)

In [4]:
embedder = Hex2VecEmbedder([15, 10, 3])

# Option 1: fit and transform
embedder.fit(regions, features, joint, neighbourhood, batch_size=128)
embeddings = embedder.transform(regions, features, joint)

# Option 2: fit_transform
# embeddings = lightning_model.fit_transform(regions, features, joint, neighbourhood, batch_size=128)

100%|██████████| 1760/1760 [00:00<00:00, 15727.48it/s]
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type       | Params | Mode 
-----------------------------------------------
0 | encoder | Sequential | 3.8 K  | train
-----------------------------------------------
3.8 K     Trainable params
0         Non-trainable params
3.8 K     Total params
0.015     Total estimated model params size (MB)
6         Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=3` reached.


In [5]:
folium_map = plot_regions(area, colormap=["rgba(0,0,0,0.1)"], tiles_style="CartoDB positron")
plot_numeric_data(regions, 1, embeddings, map=folium_map)

In [7]:
embeddings

,0,1,2
region_id,,,
8b194ad30563fff,0.655929,-0.253561,0.611199
8b194ad30005fff,0.450094,-0.163306,0.495837
8b194ad32133fff,0.343679,-0.130051,0.399245
8b194ad3284cfff,-0.579828,0.246188,-0.458590
8b194ad32155fff,0.639246,-0.249662,0.591945
...,...,...,...
8b194ad30c60fff,0.335714,-0.110411,0.401680
8b194ad362f4fff,-1.005473,0.360916,-0.991932
8b194ad30104fff,0.247166,-0.083712,0.349887


In [8]:
embeddings_vector = embeddings.to_numpy()
embeddings_vector.shape

(1760, 3)

In [11]:
embeddings_df = pd.DataFrame({
    'region': regions['geometry'],
    'embeddings': embeddings_vector.tolist()
})

In [28]:
#convert embeddings_df to gdf

embeddings_gdf = gpd.GeoDataFrame(embeddings_df, geometry='region')
embeddings_gdf.head()

,region,embeddings
region_id,,
8b194ad30563fff,"POLYGON ((-0.08463 51.51566, -0.08493 51.51552...","[0.6559289693832397, -0.2535612881183624, 0.61..."
8b194ad30005fff,"POLYGON ((-0.07729 51.51297, -0.07759 51.51283...","[0.4500935971736908, -0.16330552101135254, 0.4..."
8b194ad32133fff,"POLYGON ((-0.10945 51.51490, -0.10975 51.51476...","[0.34367862343788147, -0.1300506293773651, 0.3..."
8b194ad3284cfff,"POLYGON ((-0.09853 51.51389, -0.09883 51.51375...","[-0.5798284411430359, 0.24618752300739288, -0...."
8b194ad32155fff,"POLYGON ((-0.10673 51.51200, -0.10703 51.51186...","[0.6392459869384766, -0.24966248869895935, 0.5..."


In [ ]:
records = []
for _, row in embeddings_gdf.iterrows():
    records.append({
        'region': row['region'].wkt,  # Convert geometry to WKT format for insertion
        'embeddings': np.array(row['embeddings']).tolist()  # Ensure it's a list
    })

Save the embeddings in the GeoBase

In [13]:
from google.colab import userdata
db_string = userdata.get('db_string')

In [14]:
# Create a connection to the database
engine = create_engine(db_string)

In [24]:
create_table_sql = """
BEGIN;
CREATE TABLE embeddings_vector (
    id SERIAL PRIMARY KEY,
    region GEOMETRY(POLYGON, 4326),
    embeddings VECTOR(3)
);
COMMIT;
"""

In [38]:
with engine.connect() as connection:
    connection.execute(text(create_table_sql))

In [15]:
# Test the connection
try:
    connection = engine.connect()
    print("Connection to the database successful!")
    connection.close()
except Exception as e:
    print("Connection failed:", e)


Connection to the database successful!


In [23]:
# Define the table metadata
metadata = MetaData()

# Define the table
malmo_embeddings = Table(
    'embeddings_vector', metadata,
    Column('id', Integer, primary_key=True),
    Column('region', Geometry('GEOMETRY', srid=4326)),
    Column('embeddings', Vector(3))
)

In [ ]:
metadata.create_all(engine)

In [33]:
#save data into embeddings_vector

records = []
for _, row in embeddings_gdf.iterrows():
    records.append({
        'region': row['region'].wkt,  # Convert geometry to WKT format for insertion
        'embeddings': np.array(row['embeddings']).tolist()  # Ensure it's a list
    })
print(records[0])

{'region': 'POLYGON ((-0.0846279778008825 51.515664303296354, -0.0849267973187127 51.51552441506657, -0.0848650377660001 51.51527486497231, -0.0845044618941889 51.51516520352842, -0.0842056443069904 51.51530509133959, -0.0842674006609682 51.515554641013246, -0.0846279778008825 51.515664303296354))', 'embeddings': [0.6559289693832397, -0.2535612881183624, 0.6111989617347717]}


In [36]:
with engine.connect() as conn:
    with conn.begin():
      conn.execute(insert(malmo_embeddings), records)
      conn.commit()

In [40]:
#we can save in geobase in two ways one is

# Convert 'region' column to WKT format
# embeddings_gdf['region'] = embeddings_gdf['region'].apply(lambda geom: geom.wkt)

# Use to_sql to save the data
embeddings_gdf.to_sql('embeddings_vector', engine, if_exists='append', index=False, dtype={ 'embeddings': ARRAY(Float)})

760

In [ ]:
query = "SELECT * FROM embeddings_vector"
read_df = gpd.read_postgis(query, engine,geom_col='region')

In [ ]:
# Check the type of each column
print(read_df.dtypes)

# Verify if the 'embeddings' column can be converted to a numeric vector
import ast
try:
    read_df['embeddings'] = read_df['embeddings'].apply(ast.literal_eval)
    print("The 'embeddings' column can be converted to a numeric vector.")
except Exception as e:
    print(f"Error converting 'embeddings' column: {e}")
